# Self study 1

Self studies should be solved individually, or in small groups of 2-3 students. There is no hand-in of your solutins to the self studies. However, you can bring your solutions to the exam, and use them as the basis for your answers to the exam questions.

In this self-study we construct a simple crawler. Concretely, you should: 

* Select about 5 seed urls, e.g. homepages of universities, e-commerce sites, or similar

* Start crawling from these seeds. Define a strategy for selecting the next url to be crawled. What kind of prioritization (if any) is embodied in your strategy?

* Make sure you obey the robots.txt file, and make ensure that at least 2 seconds elapse between requests to the same host

* Stop when you have crawled approx. 1000 pages

* For each crawled page, save the url and the text string contained in the 'title' element of the document (we do not want to handle the full text of the pages at this point).

* You can repeat this several times, using different seed sets and/or prioritization strategies.

The following two self studies will extend the work that you do in this self study.

The following introduces a few helpful libraries and essential functions. You can use these methods, or use other tools that you are already familiar with and/or prefer to work with. 

A simple crawler implementation can be based on the 'requests' package [https://requests.readthedocs.io/en/master/](https://requests.readthedocs.io/en/master/) for retrieving html documents, and the BeautifulSoup parser https://www.crummy.com/software/BeautifulSoup/bs4/doc/ for parsing the html.

In [8]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from urllib.robotparser import RobotFileParser
import random
import nltk
nltk.download('punkt')
from queue import Queue

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bruger\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Let's start crawling at https://www.aau.dk/ . We first retrieve the robots.txt file and check whether we are allowed to crawl the top-level url:

In [ ]:
rp=RobotFileParser()
rp.set_url("https://www.aau.dk/")
rp.read()
if rp.can_fetch("*", "https://www.aau.dk"):
    r = requests.get("https://www.aau.dk")
    r_parse = BeautifulSoup(r.text, 'html.parser')
    print(r_parse.get_text())

In [ ]:
seed = ["https://iamafoodblog.com/", "https://pinchofyum.com/", "https://www.loveandlemons.com/","https://imdb.com/", "https://gamegrumps.com/"]
frontier = Queue()
for s in seed:
    frontier.put(s)
ps=nltk.PorterStemmer()
visited = []
results = []
while len(results) < 500:
    rp=RobotFileParser()
    p = frontier.pop(random.randint(0,len(frontier)-1))
    rp.set_url(p)
    try:
        rp.read()
    
        if rp.can_fetch("*",p):
            r=requests.get(p)
            visited.append(p)
            r_parse = BeautifulSoup(r.text, 'html.parser')
            title = r_parse.find('title')
            if title is not None:
                text = r_parse.get_text()
                tokens=nltk.word_tokenize(title)
                toks = []
                for token in tokens:
                    toks.append(ps.stem(token))
                slup = {'url': p, 'title': title.string, 'tokens': toks}
                results.append(slup)
                for a in r_parse.find_all('a'):
                    if 'href' in a.attrs:
                        l = a['href']
                        if l.startswith('https') and l not in frontier and l not in visited:
                            frontier.append(l)
    except:
        print('woops!')
print(results)
        
        

We can now get the html using the requests package, which returns a response object:

In [6]:
print(len(results))


500


In [7]:
import json


A basic view of the contents is accessible via the content attribute:

In [ ]:
ob = json.dumps(results, indent=4)
print(ob)

For serious parsing, we can use the BeautifulSoup html parser:

In [10]:
with open("crawled.json", "w") as outfile:
    json.dump(results, outfile)

In [32]:
print(len(results))


717


We can get the title:

In [6]:
print(r_parse.find('title'))
print(r_parse.find('title').string)

<title>AAU - Viden for verden</title>
AAU - Viden for verden


Importantly, we can get all the links on the page. The following also illustrates the sleep() function to implement time delays (the following will take a while to complete; use the "interrupt kernel" button to terminate early):

In [7]:
for a in r_parse.find_all('a'):
    sleep(1)
    print(a['href'])

https://www.aau.dk/nyheder
https://www.aau.dk/arrangementer
https://www.aau.dk/kontakt
https://www.aau.dk/om-aau/organisation/campus
https://www.aau.dk/pressen
https://www.alumni.aau.dk/
#
https://www.aau.dk/nyheder
https://www.aau.dk/arrangementer
https://www.aau.dk/kontakt
https://www.aau.dk/om-aau/organisation/campus
https://www.aau.dk/pressen
https://www.alumni.aau.dk/
https://www.aau.dk/uddannelser/
https://www.aau.dk/forskning/
https://www.aau.dk/samarbejde/
https://www.aau.dk/om-aau/
https://www.stillinger.aau.dk/
https://www.intern.aau.dk/
https://www.search.aau.dk/?locale=da
https://www.en.aau.dk/
https://www.search.aau.dk/?locale=da
https://www.en.aau.dk/
https://www.aau.dk
#
https://www.aau.dk/uddannelser/
https://www.aau.dk/uddannelser/bachelor
https://www.aau.dk/uddannelser/kandidat
https://www.aau.dk/uddannelser/sidefag-tilvalgsfag
https://www.aau.dk/uddannelser/bliv-gymnasielaerer
https://www.evu.aau.dk
https://www.aau.dk/uddannelser/optagelse/bachelor
http://www.adgangs